<a href="https://colab.research.google.com/github/RashikaTyagi01/Rashika-UML501/blob/main/mlAssignment5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Q1. (Based on Step-by-Step Implementation of Ridge Regression using Gradient
Descent Optimization)
Generate a dataset with atleast seven highly correlated columns and a target variable.
Implement Ridge Regression using Gradient Descent Optimization. Take different
values of learning rate (such as 0.0001,0.001,0.01,0.1,1,10) and regularization
parameter (10-15,10-10,10-5
,10- 3
,0,1,10,20). Choose the best parameters for which ridge
regression cost function is minimum and R2_score is maximum.

In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

In [4]:
np.random.seed(0)
n = 500
z = np.random.randn(n)
X = np.column_stack([z + 0.01 * np.random.randn(n) for _ in range(7)])
X = np.column_stack([X, 0.5 * z + 0.2 * np.random.randn(n)])
true_weights=np.array([-3, -2, -1, 0, 0.5, 2, 3, 4],dtype=float)
target=X.dot(true_weights) + 0.5 * np.random.randn(n)
X_train, X_test, y_train, y_test = train_test_split(X, target, test_size=0.25, random_state=1)
mean = X_train.mean(axis=0)
std = X_train.std(axis=0)
std = np.where(np.isfinite(std) & (std > 0), std, 1.0)
X_train=(X_train - mean)/std
X_test=(X_test - mean)/std

def ridge(X,y,alpha,lmbda,epochs=2000):
   X = X.astype(np.float64)
   y = y.astype(np.float64)
   m, n = X.shape
   weights = np.zeros(n, dtype=np.float64)
   bias = 0.0
   for _ in range(epochs):
        predictions = X.dot(weights) + bias
        error = predictions - y
        grad_w = (2/m)*(X.T.dot(error))+2*lmbda*weights
        grad_b = (2/m)*error.sum()
        weights -= alpha * grad_w
        bias -= alpha * grad_b
        if not (np.isfinite(weights).all() and np.isfinite(bias)):
            return None
   return weights, bias

learning_rates=[0.0001,0.001,0.01,0.1,1,10]
r_parameter=[1e-15, 1e-10, 1e-5, 1e-3, 0, 1, 10, 20]
best_r2 = -1
best_parameters = None
for lr in learning_rates:
    for lam in r_parameter:
        result = ridge(X_train, y_train, lr, lam)
        if result is None:
            continue
        weights, bias = result
        predictions = X_test.dot(weights) + bias
        if not np.isfinite(predictions).all():
            continue
        r2 = r2_score(y_test, predictions)
        if r2 > best_r2:
            best_r2 = r2
            best_parameters = (lr, lam, r2)

print("Best Learning Rate, Regularization parameter, R2 =", best_parameters)

Best Learning Rate, Regularization parameter, R2 = (0.1, 0, 0.9228808570261661)


/usr/local/lib/python3.12/dist-packages/numpy/_core/_methods.py:52: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial, where)
/usr/local/lib/python3.12/dist-packages/numpy/_core/_methods.py:52: RuntimeWarning: invalid value encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial, where)


Q2. Load the Hitters dataset from the following link
https://drive.google.com/file/d/1qzCKF6JKKMB0p7ul_lLy8tdmRk3vE_bG/view?usp=sharing
(a) Pre-process the data (null values, noise, categorical to numerical encoding)
(b) Separate input and output features and perform scaling
(c) Fit a Linear, Ridge (use regularization parameter as 0.5748), and LASSO (use
regularization parameter as 0.5748) regression function on the dataset.
(d) Evaluate the performance of each trained model on test set. Which model
performs the best and Why?

In [5]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error

In [6]:
df=pd.read_csv('/Hitters.csv')
df=df.dropna()
df=pd.get_dummies(df,columns=['League','Division','NewLeague'],drop_first=True)
X = df.drop("Salary", axis=1)
y = df["Salary"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
sc = StandardScaler()
X_train_scaled = sc.fit_transform(X_train)
X_test_scaled = sc.transform(X_test)
lr = LinearRegression()
lr.fit(X_train_scaled, y_train)
ridge = Ridge(alpha=0.5748)
ridge.fit(X_train_scaled, y_train)
lasso = Lasso(alpha=0.5748, max_iter=5000)
lasso.fit(X_train_scaled, y_train)
y_pred_lr = lr.predict(X_test_scaled)
y_pred_ridge = ridge.predict(X_test_scaled)
y_pred_lasso = lasso.predict(X_test_scaled)

print("Linear Regression R2:", r2_score(y_test, y_pred_lr))
print("Ridge Regression R2:", r2_score(y_test, y_pred_ridge))
print("Lasso Regression R2:", r2_score(y_test, y_pred_lasso))

print("\nLinear MSE:", mean_squared_error(y_test, y_pred_lr))
print("Ridge MSE:", mean_squared_error(y_test, y_pred_ridge))
print("Lasso MSE:", mean_squared_error(y_test, y_pred_lasso))


Linear Regression R2: 0.290745185579814
Ridge Regression R2: 0.30003596988293446
Lasso Regression R2: 0.29962566098567167

Linear MSE: 128284.34549672354
Ridge MSE: 126603.90264424692
Lasso MSE: 126678.11604014733


Q3. Cross Validation for Ridge and Lasso Regression
Explore Ridge Cross Validation (RidgeCV) and Lasso Cross Validation (LassoCV)
function of Python. Implement both on Boston House Prediction Dataset (load_boston
dataset from sklearn.datasets).

In [7]:
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import RidgeCV, LassoCV
from sklearn.metrics import r2_score

In [11]:

boston = fetch_openml(name="boston", version=1, as_frame=True)
X = boston.data
y = boston.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
s=StandardScaler()
X_train_scaled=s.fit_transform(X_train)
X_test_scaled=s.transform(X_test)
alphas = np.logspace(-3, 3, 50)
ridge_cv = RidgeCV(alphas=alphas, cv=5)
ridge_cv.fit(X_train_scaled, y_train)
lasso_cv = LassoCV(alphas=None, cv=5, max_iter=10000, random_state=42)
lasso_cv.fit(X_train_scaled, y_train)
print("Ridge best alpha:", ridge_cv.alpha_)
print("Ridge R2:", r2_score(y_test, ridge_cv.predict(X_test_scaled)))
print("Lasso best alpha:", lasso_cv.alpha_)
print("Lasso R2:", r2_score(y_test, lasso_cv.predict(X_test_scaled)))


Ridge best alpha: 2.6826957952797246
Ridge R2: 0.6679648354383835
Lasso best alpha: 0.006863892263379668
Lasso R2: 0.6683883969336302


Q4. Multiclass Logistic Regression: Implement Multiclass Logistic Regression (step-by step)
on Iris dataset using one vs. rest strategy?

In [9]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import accuracy_score, classification_report


In [10]:
iris_data=load_iris()
X=iris_data.data
y=iris_data.target
X_train, X_test, y_train, y_test=train_test_split(X, y, test_size=0.3, random_state=0)
sc=StandardScaler()
X_train=sc.fit_transform(X_train)
X_test=sc.transform(X_test)
model=OneVsRestClassifier(LogisticRegression(max_iter=2000))
model.fit(X_train, y_train)
y_pred=model.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.9111111111111111

Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00        16
           1       0.94      0.83      0.88        18
           2       0.77      0.91      0.83        11

    accuracy                           0.91        45
   macro avg       0.90      0.91      0.91        45
weighted avg       0.92      0.91      0.91        45

